# Pretraga

In [9]:
''' 
Lista susedstva usmerenog grafa,
sve grane imaju istu tezinu (1)

A <--1--> B --1--> C
^                  |
|                  |
---------1----------
'''

adjacency_list = {
    'A': [('B', 1)],
    'B': [('A', 1), ('C', 1)],
    'C': [('A', 1)]
}

print(adjacency_list)

{'A': [('B', 1)], 'B': [('A', 1), ('C', 1)], 'C': [('A', 1)]}


## Klasa graf

In [10]:
class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def __str__(self):
        return str(self.adjacency_list)

In [11]:
graph = Graph(adjacency_list)
print(graph)

{'A': [('B', 1)], 'B': [('A', 1), ('C', 1)], 'C': [('A', 1)]}


### Pronalaženje suseda čvora

In [12]:
class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def __str__(self):
        return str(self.adjacency_list)
    
    # Pronalazenje suseda cvora v
    def get_neighbors(self, v):
        return self.adjacency_list[v]

In [13]:
graph = Graph(adjacency_list)
print(graph.get_neighbors('A'))

[('B', 1)]


### Obilazak grafa u dubinu

In [14]:
class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def __str__(self):
        return str(self.adjacency_list)
        
    # Pronalazenje suseda cvora v
    def get_neighbors(self, v):
        return self.adjacency_list[v]
        
    # Obilazak grafa u dubinu
    def dfs(self, start, stop):
        # Inicijalizacija skupa oznacenih cvorova
        # na prazan skup
        visited = set([])
        
        # Na stek path i u skup posecenih cvorova stavi samo polazni cvor;
        visited.add(start)
        path = [start]
        
        # Dok na steku path ima elemenata
        while len(path) > 0:
            
            # Uzmi cvor n sa vrha steka path;
            n = path[-1]
            
            # Ako je cvor n ciljni cvor
            if n == stop:
                # izvesti o uspehu i vrati put konstruisan na osnovu sadrzaja steka path
                print('Pronadjen je put: {}'.format(path))
                return path
            
            # Indikator da li cvor n ima neobidjenih suseda
            has_unvisited = False
            
            # Ako n IMA potomaka koji nisu poseceni
            for (m, weight) in self.get_neighbors(n):
                if m not in visited:
                    # Izaberi prvog takvog potomka m 
                    # i dodaj ga na vrh steka path i u skup posecenih cvorova
                    path.append(m)
                    visited.add(m)
                    has_unvisited = True
                    break

            # Inace izbaci n sa steka path
            if (not has_unvisited):
                path.pop()
            
        # ako je petlja zavrsena put nije pronadjen,
        # izvesti da trazeni put ne postoji.
        print('Trazeni put ne postoji')
        return None
    

In [15]:
g = Graph(adjacency_list)
g.dfs('A','C')

Pronadjen je put: ['A', 'B', 'C']


['A', 'B', 'C']

### Obilazak grafa u širinu

In [19]:
from collections import deque

class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def __str__(self):
        return str(self.adjacency_list)
        
    def get_neighbors(self, v):
        return self.adjacency_list[v]
        
    # Obilazak grafa u sirinu
    def bfs(self, start, stop):
        visited = set([start])
        
        # Stavi samo polazni cvor u red S
        S = deque([start])
        
        # Mapa direktnih predaka cvorova
        # nadjenih prilikom obilaska
        parent = {}        
        parent[start] = start
        
        # Dok god red S nije prazan
        while len(S) > 0:
            # uzmi cvor n sa pocetka reda S i obrisi ga iz reda;
            n = S.popleft()
            
            # ako je n ciljni cvor
            if n == stop:
                path = []
                
                # Izvesti o uspehu i vrati put od polaznog do ciljnog cvora
                # (iduci unazad od ciljnog cvora);
                
                # do-while petlja ne postoji u Python-u
                while parent[n] != n:
                    path.append(n)
                    n = parent[n]

                # Dodajemo polazni cvor
                path.append(start)
                        
                path.reverse()
                print('Pronadjen je put: {}'.format(path))
                return path
            
            # za svaki od potomaka m cvora n
            for (m, weight) in self.get_neighbors(n):
                
                # za koji nije definisan roditelj
                if m not in parent:
                    
                    # zapamti n kao roditelja
                    parent[m] = n
                    
                    visited.add(m)
                    
                    # i dodaj ga na kraj reda S
                    S.append(m)
                    
        # ako je petlja zavrsena put nije pronadjen,
        # izvesti da trazeni put ne postoji.
        print('Trazeni put ne postoji')
        return None
        

In [20]:
g = Graph(adjacency_list)
print(g)

{'A': [('B', 1)], 'B': [('A', 1), ('C', 1)], 'C': [('A', 1)]}


In [21]:
g.bfs('A','C')

Pronadjen je put: ['A', 'B', 'C']


['A', 'B', 'C']


### Dijkstra algoritam

In [1]:
from heapq import heappush, heappop
from collections import deque

class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list
        
    def __str__(self):
        return str(self.adjacency_list)
        
    def get_neighbors(self, v):
        return self.adjacency_list[v]
    
    # Dijskstra
    def dijkstra(self, start, stop):

        # Q je skup ovorenih cvorova, inicijalno sadrzi sve cvorove grafa
        Q = set([v for v in self.adjacency_list])

        # D sadrzi tekuce udaljenosti od polaznog cvora (start) do ostalih cvorova, inicijalno beskonacne
        D = dict([(v, float('inf')) for v in self.adjacency_list])

        # Udaljenost polaznog cvora od samog sebe je 0
        D[start] = 0

        # Mapa parents cuva roditelje cvorova
        parent = {}
        parent[start] = start

        # Dok je skup Q neprazan:
        while len(Q) > 0:

            # Pronalazi se cvor sa najmanjoj udaljenoscu od polaznog cvora i uklanja iz Q
            n = None

            for w in Q:
                if n == None or (D[w] != float('inf') and D[w] < D[n]):
                    n = w

            # Ako ne postoji cvor cija je udaljenost manja od beskonacnosti, put od polaznog do ciljnog cvora ne postoji
            if D[n] == float('inf'):
                print('Trazeni put ne postoji')
                return None

            if n == stop:
                path = []

                # do-while petlja ne postoji u Python-u
                while parent[n] != n:
                    path.append(n)
                    n = parent[n]

                # Dodajemo polazni cvor
                path.append(start)

                path.reverse()
                
                print('Pronadjen je put: {}'.format(path))
                return path

            # proveri da li je ustanovljeno rastojanje od polaznog cvora do m vece od rastojanja
            # od polaznog cvora do m preko cvora n i ako jeste, promeniti informaciju
            # o roditelju cvora m na cvor n i upamtiti novo rastojanje.
            for (m, weight) in self.adjacency_list[n]:
                if D[m] == float('inf') or D[n] + weight < D[m]:
                    D[m] = D[n] + weight
                    parent[m] = n

            Q.remove(n)

        #  Obavesti da trazeni put ne postoji (Q je prazan skup i uspeh nije prijavljen).
        print('Trazeni put ne postoji')
        return None

In [7]:
adjacency_list = {
    'A': [('D', 100), ('C', 10), ('B', 5)],
    'B': [('C', 2)],
    'C': [],
    'D': [('C', 5)]
}

g = Graph(adjacency_list)
g.dijkstra('A', 'C')

Pronadjen je put: ['A', 'B', 'C']


['A', 'B', 'C']

## Rešavanje Lojdove slagalice

In [58]:
from collections import deque
import copy

class Game:
    def __init__(self, start_state):
        self.start_state = self.serialize(start_state)
        self.end_state = self.serialize([[1,2,3],[4,5,6],[7,8,0]])
    
    def __str__(self):
        return str(self.adjacency_list)
    
    # Kako je potrebno matricu pamtititi u mapi, neophodno je matricu prebaciti u sring
    def serialize(self, state):
        serialized = []
        for i in range(len(state)):
            for j in range(len(state[i])):
                serialized.append(str(state[i][j]))
        return ':'.join(serialized)
    
    # Matrica koja je u stringu se prebacije u regularnu matricu
    def deserialize(self, serialized):
        serialized_list = serialized.split(':')
        deserialized = [
            serialized_list[:3],
            serialized_list[3:6],
            serialized_list[6:]
        ]
        
        deserialized = [[int(x) for x in row] for row in deserialized]
        
        return deserialized
    
    # Susedi se mogu dobiti pomeranjem praznog polja u neko od 4 moguca pravca
    # Svako stanje moze imati najvise 4 suseda
    def get_neighbors(self, state):
        deserialized = self.deserialize(state)
        
        neighbors = []
        
        # Pronalazak praznog polja, odnosno 0
        blank_i = -1
        blank_j = -1
        
        for i in range(0, 3):
            for j in range(0, 3):
                if deserialized[i][j] == 0:
                    blank_i = i
                    blank_j = j
                    break
                    
        i = blank_i
        j = blank_j
              
        # Ukoliko je moguce prazno polje se pomera na gore            
        if i > 0:
                new_matrix = copy.deepcopy(deserialized)
                new_matrix[i][j] = new_matrix[i - 1][j]
                new_matrix[i - 1][j] = 0
                neighbors.append(self.serialize(new_matrix))
                
        # Ukoliko je moguce prazno polje se pomera na dole        
        if i < 2:
                new_matrix = copy.deepcopy(deserialized)
                new_matrix[i][j] = new_matrix[i + 1][j]
                new_matrix[i + 1][j] = 0
                neighbors.append(self.serialize(new_matrix))
           
        # Ukoliko je moguce prazno polje se pomera na levo
        if j > 0:
            new_matrix = copy.deepcopy(deserialized)
            new_matrix[i][j] = new_matrix[i][j - 1]
            new_matrix[i][j - 1] = 0
            neighbors.append(self.serialize(new_matrix))
            
        # Ukoliko je moguce prazno polje se pomera na desno            
        if j < 2:
                new_matrix = copy.deepcopy(deserialized)
                new_matrix[i][j] = new_matrix[i][j + 1]
                new_matrix[i][j + 1] = 0
                neighbors.append(self.serialize(new_matrix))
                
        return zip(neighbors, [1 for x in neighbors])
                
    # Obilazak grafa u dubinu
    def dfs(self):
        visited = set([self.start_state])
        path = [self.start_state]
        iterations_number = 0;
        
        while len(path) > 0:
            n = path[-1]
            
            if n == self.end_state:
                print('Pronadjen je put u {} iteracija'.format(iterations_number))
                return [self.deserialize(x) for x in path]
            
            has_neighbors = False
            
            for (m, weight) in self.get_neighbors(n):
                if m not in visited:
                    path.append(m)
                    visited.add(m)
                    has_neighbors = True
                    break
                    
            if not has_neighbors:
                path = path[:-1]
                
            iterations_number += 1
         
        print('Trazeni put nije pronadjen')
        return None
    
    # Obilazak grafa u sirinu
    def bfs(self):
        S = deque([self.start_state])
        
        parent = {}
        parent[self.start_state] = None
        iterations_number = 0
            
        while len(S) > 0:
            n = S.popleft()
            
            if n == self.end_state:
                path = []
                
                while parent[n] != None:
                    path.append(n)
                    n = parent[n]
                
                path.append(self.start_state)
                path.reverse()
                
                print('Pronadjen je put u {} iteracija'.format(iterations_number))
                return [self.deserialize(x) for x in path]
            
            for (m, weigh) in self.get_neighbors(n):
                if m not in parent:
                    S.append(m)
                    parent[m] = n
                    
            iterations_number += 1
                    
        print('Trazeni put nije pronadjen')
        return None
    
    def dijkstra(self):
        Q = set([self.start_state])
        D = {}
            
        D[self.start_state] = 0
        parent = {}
        parent[self.start_state] = None
        iterations_number = 0
        
        while len(Q) > 0:
            n = None
            minimal_distance = float('inf')
            
            for v in Q:
                if D[v] < minimal_distance:
                    n = v
                    minimal_distance = D[v]
                    
            if n == None:
                print('Put nije pronadjen! Broj iteracija {}.'.format(iterations_number))
                return None
            
            if n == self.end_state:
                path = []
                
                while parent[n] != None:
                    path.append(n)
                    n = parent[n]
                
                path.append(self.start_state)
                path.reverse()
                print('Pronadjen je put u {} iteracija'.format(iterations_number))
                return [self.deserialize(x) for x in path]
            
            for (m, weight) in self.get_neighbors(n):
                if m not in D or D[m] > D[n] + weight:
                    D[m] = D[n] + weight
                    parent[m] = n
                    Q.add(m)
                    
            Q.remove(n)
            
            iterations_number += 1
        
        print('Trazeni put nije pronadjen')
        return None

In [59]:
g = Game([
    [2,3,5],
    [1,4,0],
    [7,8,6]
])

print(g.serialize([
    [2,3,5],
    [1,4,0],
    [7,8,6]
]))

# g.dijkstra(g.start_state, g.end_state)

2:3:5:1:4:0:7:8:6


In [51]:
g.bfs() # 121 iteracija

Pronadjen je put u 121 iteracija


[[[2, 3, 5], [1, 4, 0], [7, 8, 6]],
 [[2, 3, 0], [1, 4, 5], [7, 8, 6]],
 [[2, 0, 3], [1, 4, 5], [7, 8, 6]],
 [[0, 2, 3], [1, 4, 5], [7, 8, 6]],
 [[1, 2, 3], [0, 4, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 0, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 0], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 6], [7, 8, 0]]]

In [54]:
g.dfs() # 303933 iteracija

Pronadjen je put u 303933 iteracija


[[[2, 3, 5], [1, 4, 0], [7, 8, 6]],
 [[2, 3, 0], [1, 4, 5], [7, 8, 6]],
 [[2, 0, 3], [1, 4, 5], [7, 8, 6]],
 [[2, 4, 3], [1, 0, 5], [7, 8, 6]],
 [[2, 4, 3], [1, 8, 5], [7, 0, 6]],
 [[2, 4, 3], [1, 8, 5], [0, 7, 6]],
 [[2, 4, 3], [0, 8, 5], [1, 7, 6]],
 [[0, 4, 3], [2, 8, 5], [1, 7, 6]],
 [[4, 0, 3], [2, 8, 5], [1, 7, 6]],
 [[4, 8, 3], [2, 0, 5], [1, 7, 6]],
 [[4, 8, 3], [2, 7, 5], [1, 0, 6]],
 [[4, 8, 3], [2, 7, 5], [0, 1, 6]],
 [[4, 8, 3], [0, 7, 5], [2, 1, 6]],
 [[0, 8, 3], [4, 7, 5], [2, 1, 6]],
 [[8, 0, 3], [4, 7, 5], [2, 1, 6]],
 [[8, 7, 3], [4, 0, 5], [2, 1, 6]],
 [[8, 7, 3], [4, 1, 5], [2, 0, 6]],
 [[8, 7, 3], [4, 1, 5], [0, 2, 6]],
 [[8, 7, 3], [0, 1, 5], [4, 2, 6]],
 [[0, 7, 3], [8, 1, 5], [4, 2, 6]],
 [[7, 0, 3], [8, 1, 5], [4, 2, 6]],
 [[7, 1, 3], [8, 0, 5], [4, 2, 6]],
 [[7, 1, 3], [8, 2, 5], [4, 0, 6]],
 [[7, 1, 3], [8, 2, 5], [0, 4, 6]],
 [[7, 1, 3], [0, 2, 5], [8, 4, 6]],
 [[0, 1, 3], [7, 2, 5], [8, 4, 6]],
 [[1, 0, 3], [7, 2, 5], [8, 4, 6]],
 [[1, 2, 3], [7, 0, 5], [8, 

In [60]:
g.dijkstra() # 127 iteracija

Pronadjen je put u 127 iteracija


[[[2, 3, 5], [1, 4, 0], [7, 8, 6]],
 [[2, 3, 0], [1, 4, 5], [7, 8, 6]],
 [[2, 0, 3], [1, 4, 5], [7, 8, 6]],
 [[0, 2, 3], [1, 4, 5], [7, 8, 6]],
 [[1, 2, 3], [0, 4, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 0, 5], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 0], [7, 8, 6]],
 [[1, 2, 3], [4, 5, 6], [7, 8, 0]]]